## Processing faces
In this notebook we load face image and calculate an embedding for each of the faces. Then we save them to a file.

In [1]:
import os
import glob
import cv2
import pandas as pd
import numpy as np
import face_recognition

First let's embed faces that we captured in the gazebo simulator.

In [2]:
face_folders_query = '../pictures/source_simulation/*'

# Paths to directories containing face pictures
face_folders = glob.glob(face_folders_query)
face_folders.sort()

for face_folder in face_folders:
    face_label = os.path.basename(face_folder)
    faces = glob.glob(face_folder + '/*.jpg')
    faces.sort()

    print()
    print("Calculate embeddings for images of {}".format(face_label))
    df = pd.DataFrame()
    for face_path in faces:
        face_image = face_recognition.load_image_file(face_path)

        sys.stdout.write("    {} ... ".format(os.path.basename(face_path)))

        # Calculate the encoding and append it to a dataframe
        face_encoding = face_recognition.face_encodings(face_image)
        if (len(face_encoding) > 0):
            df = df.append(face_encoding)

        print("done")
    
    # Now serialize the dataframe into a csv file
    df.to_csv('../encodings/{}.txt'.format(face_label), index=False)

..done
    image41.jpg ...done
    image42.jpg ...done
    image43.jpg ...done
    image44.jpg ...done
    image45.jpg ...done
    image46.jpg ... done
    image47.jpg ...done
    image48.jpg ...done
    image49.jpg ...done
    image50.jpg ...done
    image51.jpg ...done
    image52.jpg ...done
    image53.jpg ...done
    image54.jpg ... done
    image55.jpg ... done
    image56.jpg ... done
()
Calculate embeddings for images of face06
    image01.jpg ...done
    image02.jpg ...done
    image03.jpg ...done
    image04.jpg ...done
    image05.jpg ...done
    image06.jpg ...done
    image07.jpg ...done
    image08.jpg ...done
    image09.jpg ...done
    image10.jpg ...done
    image11.jpg ...done
    image12.jpg ...done
    image13.jpg ...done
    image14.jpg ...done
    image15.jpg ...done
    image16.jpg ...done
    image17.jpg ...done
    image18.jpg ...done
    image19.jpg ...done
    image20.jpg ...done
    image21.jpg ...done
    image22.jpg ...done
    image23.jpg ...done
    imag

Now let's process faces that we captured with camera in physical world. We first have to extract faces from the image and then obtain embeddings for them.

In [ ]:
from detectors.haar_detector import HaarDetector

# Construct a face detector from cascade data
face_detector = HaarDetector("haarcascade_face.xml")

# Get path to directory of images for each face
face_dirs = glob.glob('../pictures/source_physical_camera/*')

for face_dir in face_dirs:
    # Get label of the faces in given directory
    face_label = os.path.basename(face_dir)
    # Get all images in face directory
    images = glob.glob(face_dir + '/*.jpg')

    # Save encodings in pandas' DataFrame object and serialize it into csv
    df = pd.DataFrame()

    print("Processing images in {}".format(face_label))

    # For each image load it and find faces in it
    for ind, image_path in enumerate(images):
        print("{}/{}".format(ind+1, len(images)))
        face_detected = False
        # Read the image
        image = cv2.imread(image_path)

        # Downsample the image to find faces faster and approximate 
        # robot's camera resolution
        height, width, channels = image.shape
        new_width, new_height = (int(width / 4), int(height / 4))
        image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

        # Convert it to gray
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # First use the face_recognition's method to detect face, if it fails try
        # the Haar detector
        face_locations = face_recognition.face_locations(image)
        face_encodings = face_recognition.face_encodings(image, face_locations)
        if (len(face_encodings) > 0):
            df = df.append(face_encodings)
            face_detected = True

        # Now use Haar detector
        if not face_detected:

            # First face face locations in the grayscale image
            faces = face_detector.find_faces(gray_image)
            if len(face_locations) > 0:
                # Convert face location into proper format for face_recognition module
                face_location = (faces[0].top(), faces[0].right(), faces[0].bottom(), faces[0].left())
                face_encodings = face_recognition.face_encodings(image, [face_location])

                # Append encodings to the dataframe
                if len(face_encodings) > 0:
                    df = df.append(face_encodings)
        
    # Serialize the dataframe as csv
    df.to_csv('../encodings/{}_2.txt'.format(face_label), index=False)

In [22]:
# Let's test the encodings obtained from real images of faces (not from simulation)
# Get directories containing face images
encoding_paths = glob.glob('../encodings/face*_2.txt')
encoding_paths.sort()

# Get original images
known_faces = glob.glob('../pictures/original_faces/*.jpg')
known_faces.sort()
known_encodings = []
known_names = []

# Calculate encodings for original images
for face in known_faces:
    print("Calculating: {}".format(face))
    image = face_recognition.load_image_file(face)
    encoding = face_recognition.face_encodings(image)[0]
    name = os.path.basename(face)
    known_encodings.append(encoding)
    known_names.append(name)

known_names = [s[0:6] for s in known_names]

Calculating: ../pictures/original_faces/face01.jpg
Calculating: ../pictures/original_faces/face02.jpg
Calculating: ../pictures/original_faces/face03.jpg
Calculating: ../pictures/original_faces/face04.jpg
Calculating: ../pictures/original_faces/face05.jpg
Calculating: ../pictures/original_faces/face06.jpg
Calculating: ../pictures/original_faces/face07.jpg
Calculating: ../pictures/original_faces/face08.jpg
Calculating: ../pictures/original_faces/face09.jpg
Calculating: ../pictures/original_faces/face10.jpg
Calculating: ../pictures/original_faces/face11.jpg
Calculating: ../pictures/original_faces/face12.jpg
Calculating: ../pictures/original_faces/face13.jpg
Calculating: ../pictures/original_faces/face14.jpg
Calculating: ../pictures/original_faces/face15.jpg
Calculating: ../pictures/original_faces/face16.jpg
Calculating: ../pictures/original_faces/face17.jpg
Calculating: ../pictures/original_faces/face18.jpg
Calculating: ../pictures/original_faces/face19.jpg
Calculating: ../pictures/origin

In [25]:
# Compare all images of certain face to the original face images and recognize them
for encoding_path in encoding_paths:
    # Read all encodings of certain face into DataFrame
    df = pd.read_csv(encoding_path)
    print(os.path.basename(encoding_path))

    # Compare the encodings one by one to original faces
    for index, encoding in df.iterrows():
        face_distances = face_recognition.face_distance(known_encodings, encoding.values)
        best_match_index = np.argmin(face_distances)
        name = known_names[best_match_index]
        print("  " + name)

face01_2.txt
  face01
  face01
  face01
  face01
  face01
  face01
  face01
  face01
  face01
  face01
  face01
  face01
face02_2.txt
  face04
  face04
  face04
  face04
  face04
  face04
  face04
  face04
  face04
  face04
  face04
face03_2.txt
  face02
  face02
  face02
  face02
  face02
  face02
  face02
  face02
  face02
  face02
  face02
  face02
  face02
  face02
  face02
  face02
face04_2.txt
  face03
  face03
  face03
  face03
  face03
  face03
  face03
  face03
  face03
  face03
  face03
face05_2.txt
  face05
  face05
  face05
  face05
  face05
  face05
  face05
  face05
  face05
  face05
face06_2.txt
  face06
  face06
  face06
  face06
  face06
  face06
  face06
  face06
  face06
face07_2.txt
  face10
  face10
  face10
  face10
  face10
  face10
  face10
  face13
  face10
  face03
  face10
  face10
  face10
  face10
face08_2.txt
  face09
  face09
  face09
  face09
  face09
face09_2.txt
  face08
  face08
  face08
  face08
  face08
  face08
  face08
  face08
face10_2.txt
  face